# Projeto Porto Inteligente
#### Union Squad – Daniel Rocha, Mateus Pinheiro, Letícia Fiorito, Wallys Souza, Everton Arruda
Este projeto visa ajudar na virtualização da cadeia logística integrada dos portos brasileiros

### Importando pacotes e fazendos ajustes:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import palettable
import matplotlib.pyplot as plt
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

import matplotlib.style
import matplotlib as mpl
for m,n in zip(['grid.color','axes.titlecolor','axes.labelcolor','xtick.color','ytick.color','axes.titlesize',
    'figure.figsize','legend.framealpha'],['white','white','white','white','white','20.0','13.0, 8.0','0.8']):
    mpl.rcParams[m] = n

### Importando os dados de estadia:

In [7]:
last_scales = pd.read_csv('/Users/pedrocerbino/Downloads/ESTADIA/Últimas Escalas.csv',sep=';')
last_scales.head(2)

,Número do DUV,Porto,Data Saída do Porto
0,00007a8e030f4e27d30757b47e3beb47,SGSIN - SINGAPURA (SINGAPORE),29/12/2018 00:00
1,00007a8e030f4e27d30757b47e3beb47,NLALK - NLALK - Item de domínio recebido do Si...,14/11/2018 00:00


In [17]:
next_scales = pd.read_csv('/Users/pedrocerbino/Downloads/ESTADIA/Próximas Escalas.csv',sep=';',
                          engine='python',encoding='latin1')
next_scales.head(2)

,Número do DUV,Porto,Data Chegada no Porto
0,00007a8e030f4e27d30757b47e3beb47,BRESP - BACIA PETROLIFERA DO ESPIRITO SANTO,05/02/2019 00:00
1,0000ee254a26d88709dbd7aaf80625da,BRRIO - RIO DE JANEIRO,25/03/2017 00:00


In [9]:
stays = pd.read_csv('/Users/pedrocerbino/Downloads/ESTADIA/Estadia.csv',sep=';')
stays.head(2)

,Número do DUV,Estadia Off-Shore,Porto de estadia atual,Atracação Prevista,Atracação Efetiva,Desatracação Prevista,Desatracação Efetiva,Local(is) Atracação (área do porto > berço > cabeço),Local(is) e Data(s) Reatracação (área do porto > berço > ca,Bandeira da Embarcação,Área de Navegação,Finalidade da Embarcação,Tipo de Embarcação,Motivo de Atracação,Tipo de Viagem Chegada,Tipo de Viagem Saída,Especialidade da Carga Predominante
0,8e0879c460c1314a25bea44f73974e3e,Não,BRRIO - RIO DE JANEIRO,31/12/2016 17:30,01/01/2017 07:20,01/01/2017 15:00,01/01/2017 16:00,TERMINAL-2 MULTI-RIO > 2C21 > 255 X 266,NaN,Libéria,IMPORT/EXPORT/LONGO CURSO,Transporte de Contentores,Porta Contentor,Carga e Descarga,EXPORTAÇÃO/LONGO CURSO,IMPORTAÇÃO/LONGO CURSO,Conteiner
1,77a23e1146e03847763ea5aaa70ecbc3,Não,BRSSZ - SANTOS,01/01/2017 02:00,01/01/2017 08:30,02/01/2017 01:00,02/01/2017 03:15,Margem Esquerda (Guarujá) > EPORT II > 15/30,NaN,Libéria,IMPORT/EXPORT/LONGO CURSO,Transporte de Contentores,Porta Contentor,Carga e Descarga,EXPORTAÇÃO/LONGO CURSO,IMPORTAÇÃO/LONGO CURSO,Conteiner


### Importando os dados de cargas:

In [16]:
loose_load = pd.read_csv('/Users/pedrocerbino/Downloads/CARGAS/CARGAS_SOLTAS.csv',nrows=100000,
                        engine='python',encoding='latin1')
loose_load.head(2)

,Número do DUV,Número do Conhecimento,Tipo de conhecimento,Tipo de tráfego,País de origem da mercadoria,Porto de origem da mercadoria,País de destino da mercadoria,Porto de destino da mercadoria,País de procedência da carga,Corrente de tráfego,Categoria da carga,Número do Item,Tipo,Peso total bruto,NCM Mercadoria,Peso bruto da mercadoria,Cubagem em m3
0,NaN,abfe4a1149e5bb7e3ef45568ecda0267,Filhote,Longo Curso Importação - LCI,China,CNYTN - YANTIAN,NaN,BRSSZ - SANTOS,China,NaN,Importada,941926b90cc00f04362aedccc41a2875,CARGA_SOLTA,1508.2,8531,754.100,10.89
1,NaN,44fbf353ffaafa4e007f2a2b7602ce05,Filhote,Longo Curso Importação - LCI,China,CNZGA - ZHONGSHAN,NaN,BRITJ - ITAJAÍ,China,NaN,Importada,1c2d0881ef5f2accb068e581ca249127,CARGA_SOLTA,79.4,4002,0.756,0.65


In [10]:
loose_load_date = pd.read_csv('/Users/pedrocerbino/Downloads/CARGAS/Data_Carga_Solta.csv')#,sep=';')
loose_load_date.head(2)

,Número do Conhecimento,Data de Emissão do Conhecimento,Número do Item,Tipo,Data de Registro do Item de Carga
0,65408df1cb57e145952b33f1793fc3c6,2019-07-07,e5e794691ac311554c59da9bc13fe214,CARGA_SOLTA,2019-07-22 15:13:30.577361-03
1,185b6d9ce8e0c6db53b830e9ed48185f,2019-08-23,5cf9e52f527e9e68e04bb8df61058f00,CARGA_SOLTA,2019-09-09 14:01:27.410294-03


In [11]:
granel_date = pd.read_csv('/Users/pedrocerbino/Downloads/CARGAS/Data_Granel.csv')
granel_date.head(2)

,Número do Conhecimento,Data de Emissão do Conhecimento,Número do Item,Tipo,Data de Registro do Item de Carga
0,d3e03f2b45485748c41005e25b4cf6e2,2018-10-12,dc363e2f13e0752681703d330070fc00,GRANEL,2018-10-22 16:32:32.556772-03
1,af12e2dc9241b6031642622c9cf5945b,2018-11-30,f673179b9be76365cd328af5f6fa440b,GRANEL,2018-11-23 23:14:47.202546-02


In [15]:
granel = pd.read_csv('/Users/pedrocerbino/Downloads/CARGAS/GRANEL.csv',engine='python'
                     ,encoding='latin1')
granel.head(2)

,Número do DUV,Número do Conhecimento,Tipo de conhecimento,Tipo de tráfego,País de origem da mercadoria,Porto de origem da mercadoria,País de destino da mercadoria,Porto de destino da mercadoria,País de procedência da carga,Corrente de tráfego,Categoria da carga,Número do Item,Tipo,Peso total bruto,NCM Mercadoria,Peso bruto da mercadoria,Cubagem em m3
0,NaN,a6d05acd7045da1c1709dc375f5d4e76,Comum,Interior - ITR,NaN,BRMAO - MANAUS,NaN,BR095 - TAPAUÁ,NaN,NaN,Nacional,e9c5e3817b0c1f7c9bf8b58617e33c0e,GRANEL,1.539000e+05,27101921.0,1.539000e+05,NaN
1,NaN,929f55597cff49311d8e85b340f84b9f,Comum,Cabotagem - CAB,NaN,BRSAS - BACIA PETROLIFERA DE SANTOS,NaN,BRRIO - RIO DE JANEIRO,NaN,NaN,Nacional,25bce01387542c3a11392e81ccb8d062,GRANEL,3.482801e+07,2709.0,3.482801e+07,39541.332


In [14]:
vehicle = pd.read_csv('/Users/pedrocerbino/Downloads/CARGAS/VEICULO.csv',nrows=100000,
                      engine='python',encoding='latin1')
vehicle.head(2)

,Número do DUV,Número do Conhecimento,Tipo de conhecimento,Tipo de tráfego,País de origem da mercadoria,Porto de origem da mercadoria,País de destino da mercadoria,Porto de destino da mercadoria,País de procedência da carga,Corrente de tráfego,Categoria da carga,Número do Item,Tipo,Peso total bruto,NCM Mercadoria,Peso bruto da mercadoria,Cubagem em m3
0,NaN,d7cf3f43557a06bf87d846d02c2b5cce,Comum,Longo Curso Exportação - LCE,NaN,BRSSZ - SANTOS,Argentina,ARZAE - ZARATE,NaN,NaN,Exportada,d347d199d177110962c851a02bc84cd1,VEICULO,1009.249,8703.0,1009.249,NaN
1,df46dd0a2db6036b51e6c3654b8461ca,71b68aa3bfda0fae2ccaa435160f4d51,Comum,Longo Curso Exportação - LCE,NaN,BRSSZ - SANTOS,Suíça,CLSAI - SAN ANTONIO,NaN,NaN,Exportada,c31c13b49b3e26d3753cd85ba69ec621,VEICULO,1110.000,8703.0,1110.000,NaN


In [13]:
vehicle_date = pd.read_csv('/Users/pedrocerbino/Downloads/CARGAS/Data_Veiculo.csv')
vehicle_date.head(2)

,Número do Conhecimento,Data de Emissão do Conhecimento,Número do Item,Tipo,Data de Registro do Item de Carga
0,1d49d8180cb6db0b7a8a51051d0c1a84,2019-09-30,a2d142735b0c6ec45ce85fcd511aaaeb,VEICULO,2019-10-31 18:52:41.714751-03
1,1d49d8180cb6db0b7a8a51051d0c1a84,2019-09-30,9248ab9f59df4788e039bc82b20f033d,VEICULO,2019-10-31 18:52:41.714751-03
